In [ ]:
import psycopg2
import pandas.io.sql as psql
import json
from google.cloud import bigquery
import gspread
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

#establishing the connection
def get_data(input_db_name):
    js = json.load(open("postgres_info.json"))
    db_name = input_db_name
    conn = psycopg2.connect(
       database=db_name, user=js['user'], password=js['password'], host=js['host'], port= js['port']
    )

    query_table = '''
    select *
    from information_schema.tables
    where 1=1
    and table_schema = 'public'
    '''
    df_name = psql.read_sql(query_table, conn)
    for table in df_name["table_name"]:
            query_data = '''
            select *
            from {}
            '''.format(table)
            df = psql.read_sql(query_data, conn)

            schema = []
            new_columns = []
            for name in df.columns:
                schema.append(bigquery.SchemaField("_".join(name.split(" ")), bigquery.enums.SqlTypeNames.STRING))
                new_columns.append("_".join(name.split(" ")))
            df.columns = new_columns
            result = df.to_json(orient="records")
            json_list = json.loads(result)
            for i in json_list:
                for key in i.keys():
                    i[key] = str(i[key])
            job_config = bigquery.LoadJobConfig(schema=schema, write_disposition="WRITE_TRUNCATE")
            client = bigquery.Client.from_service_account_json('google_token.json')
            prefix = 'taptap-c2c5f.manual_dwh.postgres_'
            table_id = prefix + db_name + "__" + table
            print(table_id)
            job = client.load_table_from_json(
                json_list, table_id, job_config=job_config
            ) 
            print(job.result())

gc = gspread.service_account("gg_key.json")
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1FD4Pz61dKjK4Kc5KU5gw93V6-kGeOd_eeIMVYcbJW2M/edit#gid=0")
worksheet = sh.get_worksheet(2)
df_sheet = pd.DataFrame(worksheet.get_all_records())
for i in range(df_sheet.shape[0]):
#     cl = df_sheet.iloc[i]['collection_name']
    db = df_sheet.iloc[i]['db_name']
    get_data(db)

In [6]:
import json
js = json.load(open("postgres_info.json"))

In [7]:
js

{'database': 'postgres',
 'user': 'vu.d.ngo',
 'password': 'AkSrsSlDbTfl7rLiSZ',
 'host': 'pr-rds-postgres-01.ce8gqoct0v1p.ap-southeast-1.rds.amazonaws.com',
 'port': '5432'}

In [8]:
js['database']

'postgres'

In [ ]:
import psycopg2 as pg
import pandas.io.sql as psql
connection = pg.connect("host=localhost dbname=kinder user=your_username password=your_password")
dataframe = psql.read_sql('SELECT * FROM product_product', connection)
product_category = psql.read_sql_query('select * from product_category', connection)